In [1]:
# Use GPU
!pip install timm spikingjelly pyyaml tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
!git clone https://github.com/Fat-ctrl/QKFormer.git

Cloning into 'QKFormer'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 340 (delta 101), reused 46 (delta 19), pack-reused 156 (from 1)
Receiving objects: 100% (340/340), 1.98 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [3]:
import sys
sys.path.append("./QKFormer/imagenet")

In [4]:
# Download weight
!gdown --fuzzy https://drive.google.com/file/d/1-X75OSAsALW-m7ukm_eBiTPTrYS5qclp/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1-X75OSAsALW-m7ukm_eBiTPTrYS5qclp
From (redirected): https://drive.google.com/uc?id=1-X75OSAsALW-m7ukm_eBiTPTrYS5qclp&confirm=t&uuid=57684b4c-31f4-4600-97ec-85f06b888c82
To: /content/checkpoint-203.pth
100% 780M/780M [00:12<00:00, 60.3MB/s]


In [5]:
import torch
from QKFormer.imagenet.qkformer import QKFormer_10_768  # or QKFormer_10_512, QKFormer_10_768

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = QKFormer_10_768(T=4)  # Set T to match your training
model.to(device)
model.eval()

# Load checkpoint
checkpoint = torch.load('./checkpoint-203.pth', map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model'])

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


<All keys matched successfully>

In [6]:
# Download test image, use whatever image you perfer
!wget https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/refs/heads/master/n01514668_cock.JPEG

--2025-05-27 14:59:30--  https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/refs/heads/master/n01514668_cock.JPEG
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122371 (120K) [image/jpeg]
Saving to: ‘n01514668_cock.JPEG’

n01514668_cock.JPEG 100%[===================>] 119.50K  --.-KB/s    in 0.01s   

2025-05-27 14:59:30 (8.96 MB/s) - ‘n01514668_cock.JPEG’ saved [122371/122371]



In [7]:
from PIL import Image
from torchvision import transforms

img_path = './n01514668_cock.JPEG'
img = Image.open(img_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_tensor = transform(img).unsqueeze(0).to(device)

In [8]:
with torch.no_grad():
    output = model(img_tensor)
    pred = output.argmax(dim=1).item()
print(f"Predicted class: {pred}")

Predicted class: 7


In [9]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2025-05-27 14:59:31--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2025-05-27 14:59:32 (99.4 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



Check result in imagenet_classes.txt